<img src="img/logo.svg" width="400" align="right"><br><img src="img/pangaea-logo.png" width="150" align="right">

# ICOS Carbon Portal
# PANGAEA
    

# Example: Multisource


## Documentation

Full documentation for the ICOS library on the [project page](https://icos-carbon-portal.github.io/pylib/), how to install and wheel on [pypi.org](https://pypi.org/project/icoscp/), source available on [github](https://github.com/ICOS-Carbon-Portal/pylib).


The Pangaea library is available on [pypi.org](https://pypi.org/project/pangaeapy/), source and documentation [github]
(https://github.com/pangaea-data-publisher/pangaeapy).


## Import libraries

In [ ]:
# Import some standard libraries
import matplotlib.pyplot as plt
import folium

import cartopy
cartopy.config['data_dir'] = '/data/project/cartopy/'
import cartopy.crs as ccrs
import cartopy.feature as feat

# Import ICOS and PANGAEA library
from icoscp.cpb.dobj import Dobj
from pangaeapy import PanDataSet

# A technical note: Due to an upcoming change of the package shapely 
# we suppress warnings they deliver in ShapelyDeprecationWarning
# (see the example of code cell 7 of this notebook)
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore",category=ShapelyDeprecationWarning)


## Multi Source

This example highlights the possibility to use two independent data sets. One is loaded from the ICOS Carbon Portal, ocean domain and contains data from a boat traveling from Europe to Brazil. The second data set contains similar data, collected from another boat travelling from Europe to South America.

This example was created as demonstrator and in support for <b>"Integrating data and analysis technologies within leading environmental research infrastructures: Challenges and approaches"</b>, Huber et al., 2021,  [https://doi.org/10.1016/j.ecoinf.2021.101245](https://doi.org/10.1016/j.ecoinf.2021.101245)

## Loading data from ICOS and PANGAEA

In [ ]:
icosdata = Dobj('https://meta.icos-cp.eu/objects/xgu4rfCmqvXb4w1wGGD6mYsB')
icosdata_frame = icosdata.get()

pandata = PanDataSet('10.1594/PANGAEA.889516')
pandata_frame=pandata.data

## Lets have a look at the data returned

In [ ]:
icosdata_frame.head(3)

In [ ]:
pandata_frame.head(3)

## Plotting water temperature
Now we can plot the water temperature from both campaigns into one plot. You will see the rise in temperature while traveling through the equator.

In [ ]:
plt.figure(figsize=(12,12))
ax = plt.axes()
icosdata_frame.plot(x='TIMESTAMP', y='Temp [degC]',ax=ax,linestyle='None', marker='o',grid='on',legend=True)
pandata_frame.plot(x='Date/Time', y='Temp',ax=ax, linestyle='None', marker='o',grid='on')
plt.show()

## Interactive map (geolocation)
To make more sense, of the data plot, we probably want to get a map. Using folium, which is the python interface to leaflet, we get a zoomable map, we can add different basemaps etc.

In [ ]:
# Extract the cooridnates from both datasets
icos_coord = list(zip(icosdata_frame.Latitude, icosdata_frame.Longitude))
pan_coord = list(zip(pandata_frame.Latitude, pandata_frame.Longitude))

# Create the map with two layers of tiles
myMap = folium.Map(location=[20, 0], zoom_start=2)
folium.TileLayer('openstreetmap').add_to(myMap)
folium.TileLayer('stamenterrain').add_to(myMap)

# add dataframe coordinates
folium.PolyLine(locations = icos_coord,color='red').add_to(myMap)
folium.PolyLine(locations = pan_coord, color='black').add_to(myMap)
folium.LayerControl().add_to(myMap)
myMap


## Map, geolocation, colourcoded
This time lets use cartopy and matplotlib, to create a figure displaying the boats movement where water temperature and location is combined. The water temperature is colour coded to the geolocation.

In [ ]:
proj=ccrs.cartopy.crs.Miller()
plt.figure(dpi=150)
ax = plt.axes(projection=proj)
ax.set_extent([-60, 20, 40, -40])
ax.stock_img()
land_50m = feat.NaturalEarthFeature('physical', 'land', '50m',edgecolor='grey',facecolor=feat.COLORS['land'])
ax.add_feature(land_50m)
ax.coastlines()

# Note: Here is the code that gives deprecation warnings as mentioned in the beginning
ax.scatter(pandata_frame['Longitude'],pandata_frame['Latitude'],c=pandata_frame['Temp'],s=0.2,transform=ccrs.PlateCarree())
ax.scatter(icosdata_frame['Longitude'],icosdata_frame['Latitude'],c=icosdata_frame['Temp [degC]'],s=0.3,transform=ccrs.PlateCarree())
plt.show()

## Citing the data set

In [ ]:
for references in [icosdata.citation, pandata.citation]:
    print(references)